In [1]:
import pyoptflight as pof
from pyoptflight import initialize as optinit
from pyoptflight import plotting as optplot
import numpy as np

In [2]:
kerbin = pof.Body("Kerbin")

# vehicle = pof.Stage.load_vehicle('mintoc_multi')
vehicle = pof.Stage.load_vehicle('mintoc_single')

config = pof.SolverConfig(landing=False, 
                          T_min=1,
                          T_max = 700,
                          max_iter=250, 
                          solver_tol=1e-4, 
                          N=100, 
                          T_init=100,
                          q_max = 100,
                          integration_method='RK4')

x0 = pof.LatLngBound(lat=0, lng=0, alt=0, vel = 1e-6, ERA0=0)

xf = pof.KeplerianBound(i=np.deg2rad(60),
                           Ω=np.deg2rad(15),
                           ω=0,
                           ha=80,
                           hp=80,
                        #    ν = np.deg2rad(100),
                           body=kerbin)

# single_stage_solver = pof.Solver(kerbin, [single_stage], config, x0, xf)

msolver = pof.Solver(kerbin, vehicle, config, x0, xf)

In [3]:
msolver.create_nlp_3()

In [ ]:
msolver.stats()

In [ ]:
msolver.initialize_from_func(pof.gravity_turn, opts={'skew':True})

In [ ]:
xf = pof.StateBound(state=msolver.sols[0][-1].X[-1][1:10])
msolver.xf = xf
np.linalg.norm(msolver.sols[0][-1].X[-1][1:4])

In [ ]:
# msolver.extra_opts = {'solver': 'fatrop'}
msolver.extra_opts = {'solver': 'ipopt'}

In [ ]:
msolver.create_nlp_2()
msolver.solve_nlp_2()
msolver.stats()

In [ ]:
msolver_full = pof.Solver(kerbin, vehicle, config, x0, xf)
msolver_full.extra_opts = {'solver': 'fatrop'}
msolver_full.initialize_from_func(pof.gravity_turn, opts={'skew':True})
msolver_full.create_nlp()
msolver_full.solve_nlp()
msolver_full.stats()

In [ ]:
msolver_full.extra_opts = {'solver': 'ipopt'}
msolver_full.create_nlp()
msolver_full.solve_nlp()
msolver_full.stats()

In [ ]:
pof.plot_solutions(msolver, colorscale='vel', markers=None, show_actual_orbit=True, show_target_orbit=True, indices=[1, 3])

In [ ]:
np.max(np.abs(np.array(msolver.sols[-1][0].U)[:, 2]))

In [ ]:
np.array(msolver.sols[-1][-1].X)[:, 0]

In [ ]:
ν_range = (-np.pi, np.pi)
coord_func = lambda ν: pof.functions.kep_to_state(0, 680, 0, 0, 0, ν[0], kerbin.mu)[0][0]
pof.boundary_objects._optimize_extreme(coord_func, [ν_range])

In [ ]:
coord_func = lambda ν: pof.functions.kep_to_state(0, 680, 0, 0, 0, ν, kerbin.mu)[0][0]
coord_func(0)

In [ ]:
import numpy as np
r = np.arange(0, 9)
R = np.array([[r, r+1, r+2], [r+3, r+4, r+5], [r+6, r+7, r+8]])
a = np.array([r, r, r]).T
for i in range(9):
    print(R[:,:,i], a[i])
print(R.shape)
R = np.moveaxis(R, -1, 0)  # Now R.shape is (9, 3, 3)
print(R.shape)
rotated_data = np.matmul(R, a[..., np.newaxis]).squeeze(-1)
rotated_data

In [ ]:
np.transpose(R, axes=(0, 2, 1))

In [ ]:
A = np.arange(36).reshape((-1,3))
blocks = np.array([4, 4, 4])
cum_blocks = np.cumsum(blocks)
insert_values = np.vstack((A[cum_blocks[:-1]], A[-1]))
# A_ins = np.insert(A, cum_blocks, np.zeros(3), axis=0)
# A_ins[cum_blocks+np.arange(3)] = insert_values
A_ins = np.insert(A, cum_blocks, insert_values, axis=0)

In [ ]:
A_ins

In [ ]:
R = np.arange(9).reshape((3, 3))
(R @ A_ins.T).T

In [ ]:
dir_vec = np.array([0, 2**0.5, 2**0.5])
mag = np.linalg.norm(dir_vec)
np.rad2deg(np.arcsin(-dir_vec[2]/mag))

In [ ]:
np.block([np.arange(15)[:, np.newaxis], A_ins, A_ins])

In [ ]:
len(A_ins)

In [ ]:
N = [10, 10, 5]
N_arr = np.array(N)
nx = 10
nu = 3
np.concatenate(([0], np.cumsum((N_arr+1)*(nx+1) + N_arr*nu)))

In [ ]:
(N_arr+1)*(nx+1) + N_arr*nu